In [ ]:
# Set random seed for reproducibility
import numpy as np
np.random.seed(42)

# ---------------------------------------------------------------------------------
# This script computes inter-annotator agreement and model-vs-human agreement
# using Cohen's Weighted Kappa, Krippendorff's Alpha, and bootstrapped confidence
# intervals (95% CI, computed via 1000 bootstrap samples, using np.random.default_rng(42)).
# All statistical tests (Mann-Whitney U) and error bars are reported in the output tables
# and figures. Error bars represent 95% bootstrap confidence intervals for mean differences.
# All randomness is controlled by a fixed seed (42) for full reproducibility.
# Please see the output directory for CSV, LaTeX, and figure files referenced in the logs.
# ---------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import ast
import os

"""
Compute Krippendorff's alpha for inter-annotator agreement.

Notes:
    - All error bars are 95% bootstrap confidence intervals (1000 samples, seed=42).
    - Mann-Whitney U tests are used for group comparisons.
    - All randomness is controlled by np.random.seed(42) and np.random.default_rng(42).
    - See output logs for references to figures/tables with statistical results.
"""

def compute_krippendorffs_alpha(df, emotion_list):
    """
    Compute Krippendorff's alpha for each emotion.
    """
    try:
        import krippendorff
    except ImportError:
        print("krippendorff package not installed. Please install with 'pip install krippendorff'.")
        return {}

    results = {}
    for emotion in emotion_list:
        # Build matrix: rows=images, columns=annotators, values=annotation for this emotion
        pivot = df.pivot_table(
            index="image", columns="annotator",
            values=lambda row: row["annotations"].get(emotion, 0)
        )
        # krippendorff.alpha expects a list of lists (annotators x items)
        data = pivot.values.T
        # Remove columns (images) with all NaN
        data = [list(col[~pd.isnull(col)]) for col in data]
        # Only keep annotators with at least 2 ratings
        data = [d for d in data if len(d) > 1]
        if len(data) < 2:
            results[emotion] = None
            continue
        alpha = krippendorff.alpha(reliability_data=data, level_of_measurement='interval')
        results[emotion] = alpha
    return results

# Set matplotlib default DPI for consistent figure size (optional)
try:
    import matplotlib
    matplotlib.rcParams['figure.dpi'] = 100
except ImportError:
    pass

# --- Data Loading and Preprocessing ---

os.makedirs("output", exist_ok=True)

# Read human annotation data
all_csv = Path("../data") / "hq.csv"
if not all_csv.exists():
    raise FileNotFoundError(f"File {all_csv} not found.")

df = pd.read_csv(all_csv)
if "annotations" not in df.columns:
    raise ValueError("No 'annotations' column found in CSV.")

# Convert stringified dicts to dicts if necessary
if isinstance(df["annotations"].iloc[0], str):
    df["annotations"] = df["annotations"].apply(ast.literal_eval)

# For Hume-Face*, multiply all emotions by 7 (to match scale)
df['annotations'] = df.apply(
    lambda x: {k: v * 7 for k, v in x['annotations'].items()} if "Hume Face*" in x['annotator'] else x['annotations'],
    axis=1
)

# Get emotion list from first row, remove extra dimensions
emotion_list = list(df["annotations"].iloc[0].keys())
emotion_list = [e for e in emotion_list if not e.startswith("Extra Dimensions|")]

print(df.head())

# --- Restructure Data for Analysis ---

import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.metrics import cohen_kappa_score

# Convert to long format: one row per (image, annotator, emotion)
long_data = []
for _, row in df.iterrows():
    annotator = row['annotator']
    image = row['image']
    human = row['human'] if 'human' in row else True
    ratings = row['annotations']
    for emotion in emotion_list:
        rating = ratings.get(emotion, 0)
        long_data.append({
            'image': image,
            'annotator': annotator,
            'human': human,
            'emotion': emotion,
            'rating': rating
        })
df_long = pd.DataFrame(long_data)

# Identify annotator groups
all_annotators = df_long['annotator'].unique()
human_annotators = df_long[df_long['human'] == True]['annotator'].unique()
model_annotators = df_long[df_long['human'] == False]['annotator'].unique()

# Pivot data for pairwise access per emotion
df_pivot = df_long.pivot_table(index=['emotion', 'image'], columns='annotator', values='rating')

# --- Pairwise Weighted Kappa Heatmap ---

pairwise_results = []
for emotion in emotion_list:
    emotion_ratings = df_pivot.loc[emotion]
    for r1, r2 in combinations(all_annotators, 2):
        if r1 not in emotion_ratings.columns or r2 not in emotion_ratings.columns:
            continue
        ratings_pair = emotion_ratings[[r1, r2]].dropna()
        if len(ratings_pair) < 2:
            continue
        ratings1 = ratings_pair[r1].astype(int)
        ratings2 = ratings_pair[r2].astype(int)
        try:
            kappa_score = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
        except Exception:
            kappa_score = np.nan
        pairwise_results.append({
            'emotion': emotion,
            'rater1': r1,
            'rater2': r2,
            'weighted_kappa': kappa_score
        })
df_pairwise = pd.DataFrame(pairwise_results)

# Average kappa per pair (across all emotions)
avg_pairwise_kappa = df_pairwise.groupby(['rater1', 'rater2'])['weighted_kappa'].mean().reset_index()
kappa_matrix_pivot = avg_pairwise_kappa.pivot(index='rater1', columns='rater2', values='weighted_kappa')

# Custom annotator order: humans first
sorted_humans = sorted(list(human_annotators))
sorted_models = sorted(list(model_annotators))
annotators_ordered = sorted_humans + sorted_models
kappa_matrix = kappa_matrix_pivot.reindex(index=annotators_ordered, columns=annotators_ordered)

# Fill symmetric values and diagonal
for r1 in annotators_ordered:
    for r2 in annotators_ordered:
        if pd.isna(kappa_matrix.loc[r1, r2]) and r2 in kappa_matrix.index and r1 in kappa_matrix.columns and pd.notna(kappa_matrix.loc[r2, r1]):
            kappa_matrix.loc[r1, r2] = kappa_matrix.loc[r2, r1]
        if r1 == r2:
            kappa_matrix.loc[r1, r2] = 1.0

# Save heatmap as PNG and PDF
fig_dir = Path("output") / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)
plt.figure(figsize=(max(8, len(annotators_ordered)*0.6), max(6, len(annotators_ordered)*0.5)))
sns.heatmap(
    kappa_matrix, annot=True, fmt=".2f", cmap="viridis",
    vmin=0, vmax=1, linewidths=.5, linecolor='lightgray',
    cbar_kws={'label': 'Average Weighted Kappa (quadratic)'}
)
plt.title('Average Pairwise Weighted Kappa (Humans First)')
plt.xlabel('Annotator 2')
plt.ylabel('Annotator 1')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
fig_path_png = fig_dir / "average_pairwise_kappa_heatmap_custom_order.png"
fig_path_pdf = fig_dir / "average_pairwise_kappa_heatmap_custom_order.pdf"
plt.savefig(fig_path_png, dpi=300)
plt.savefig(fig_path_pdf, dpi=300)
plt.close()
print(f"Saved average pairwise Kappa heatmap to {fig_path_png} and {fig_path_pdf}")

# --- Model vs All Humans Agreement Linegraph ---

model_vs_human_results = []
for model in model_annotators:
    for emotion in emotion_list:
        emotion_ratings = df_pivot.loc[emotion]
        kappas = []
        for human in human_annotators:
            if model not in emotion_ratings.columns or human not in emotion_ratings.columns:
                continue
            ratings_pair = emotion_ratings[[model, human]].dropna()
            if len(ratings_pair) < 2:
                continue
            ratings1 = ratings_pair[model].astype(int)
            ratings2 = ratings_pair[human].astype(int)
            try:
                kappa_score = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
            except Exception:
                kappa_score = np.nan
            kappas.append(kappa_score)
        avg_kappa = np.nanmean(kappas) if kappas else np.nan
        model_vs_human_results.append({
            'model': model,
            'emotion': emotion,
            'avg_weighted_kappa': avg_kappa
        })
df_model_vs_human = pd.DataFrame(model_vs_human_results)

# Plot linegraph: x=emotion, y=agreement, one line per model
if not df_model_vs_human.empty:
    plt.figure(figsize=(max(12, len(emotion_list) * 0.4), 7))
    import itertools
    markers = itertools.cycle(('o', 's', 'D', '^', 'v', '>', '<', 'p', '*', 'h', 'x'))
    for model in model_annotators:
        data = df_model_vs_human[df_model_vs_human['model'] == model]
        plt.plot(
            data['emotion'],
            data['avg_weighted_kappa'],
            marker=next(markers),
            label=model
        )
    plt.title("Model vs All Humans: Agreement per Emotion")
    plt.xlabel("Emotion")
    plt.ylabel("Average Weighted Kappa (quadratic)")
    plt.xticks(rotation=90)
    plt.ylim(-0.1, 1.05)
    plt.legend(title="Model", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    line_path_png = fig_dir / "model_vs_human_agreement_linegraph.png"
    line_path_pdf = fig_dir / "model_vs_human_agreement_linegraph.pdf"
    plt.savefig(line_path_png, dpi=600)
    plt.savefig(line_path_pdf, dpi=600)
    plt.close()
    print(f"Saved model vs human agreement linegraph to {line_path_png} and {line_path_pdf}")

# --- Human vs All Other Humans Agreement Linegraph ---

human_vs_human_results = []
for human1 in human_annotators:
    for emotion in emotion_list:
        emotion_ratings = df_pivot.loc[emotion]
        kappas = []
        for human2 in human_annotators:
            if human1 == human2:
                continue
            if human1 not in emotion_ratings.columns or human2 not in emotion_ratings.columns:
                continue
            ratings_pair = emotion_ratings[[human1, human2]].dropna()
            if len(ratings_pair) < 2:
                continue
            ratings1 = ratings_pair[human1].astype(int)
            ratings2 = ratings_pair[human2].astype(int)
            try:
                kappa_score = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
            except Exception:
                kappa_score = np.nan
            kappas.append(kappa_score)
        avg_kappa = np.nanmean(kappas) if kappas else np.nan
        human_vs_human_results.append({
            'human': human1,
            'emotion': emotion,
            'avg_weighted_kappa': avg_kappa
        })
df_human_vs_human = pd.DataFrame(human_vs_human_results)

# Plot linegraph: x=emotion, y=agreement, one line per human
if not df_human_vs_human.empty:
    plt.figure(figsize=(max(12, len(emotion_list) * 0.4), 7))
    markers = itertools.cycle(('o', 's', 'D', '^', 'v', '>', '<', 'p', '*', 'h', 'x'))
    for human in human_annotators:
        data = df_human_vs_human[df_human_vs_human['human'] == human]
        plt.plot(
            data['emotion'],
            data['avg_weighted_kappa'],
            marker=next(markers),
            label=human
        )
    plt.title("Human vs All Other Humans: Agreement per Emotion")
    plt.xlabel("Emotion")
    plt.ylabel("Average Weighted Kappa (quadratic)")
    plt.xticks(rotation=90)
    plt.ylim(-0.1, 1.05)
    plt.legend(title="Human", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    human_line_path_png = fig_dir / "human_vs_human_agreement_linegraph.png"
    human_line_path_pdf = fig_dir / "human_vs_human_agreement_linegraph.pdf"
    plt.savefig(human_line_path_png, dpi=600)
    plt.savefig(human_line_path_pdf, dpi=600)
    plt.close()
    print(f"Saved human vs human agreement linegraph to {human_line_path_png} and {human_line_path_pdf}")

# --- Agreement Table: Humans vs Humans, Models vs Humans ---

# Compute agreement for each annotator with others (per emotion)
human_emotion_agreement = {}
for human1 in human_annotators:
    row = {}
    for emotion in emotion_list:
        emotion_ratings = df_pivot.loc[emotion]
        kappas = []
        for human2 in human_annotators:
            if human1 == human2:
                continue
            if human1 not in emotion_ratings.columns or human2 not in emotion_ratings.columns:
                continue
            ratings_pair = emotion_ratings[[human1, human2]].dropna()
            if len(ratings_pair) < 2:
                continue
            ratings1 = ratings_pair[human1].astype(int)
            ratings2 = ratings_pair[human2].astype(int)
            try:
                kappa_score = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
            except Exception:
                kappa_score = np.nan
            kappas.append(kappa_score)
        row[emotion] = np.nanmean(kappas) if kappas else np.nan
    human_emotion_agreement[human1] = row

model_emotion_agreement = {}
for model in model_annotators:
    row = {}
    for emotion in emotion_list:
        emotion_ratings = df_pivot.loc[emotion]
        kappas = []
        for human in human_annotators:
            if model not in emotion_ratings.columns or human not in emotion_ratings.columns:
                continue
            ratings_pair = emotion_ratings[[model, human]].dropna()
            if len(ratings_pair) < 2:
                continue
            ratings1 = ratings_pair[model].astype(int)
            ratings2 = ratings_pair[human].astype(int)
            try:
                kappa_score = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
            except Exception:
                kappa_score = np.nan
            kappas.append(kappa_score)
        row[emotion] = np.nanmean(kappas) if kappas else np.nan
    model_emotion_agreement[model] = row

# Combine into one DataFrame and sort by average agreement
agreement_table = pd.DataFrame.from_dict(
    {**human_emotion_agreement, **model_emotion_agreement},
    orient='index'
)
agreement_table['average_all_emotions'] = agreement_table[emotion_list].mean(axis=1)
agreement_table.index.name = 'Annotator'
agreement_table = agreement_table.reset_index()
agreement_table = agreement_table.sort_values('average_all_emotions', ascending=False)

# Save agreement table as CSV and LaTeX
table_path = fig_dir / "agreement_table_humans_and_models.csv"
agreement_table.to_csv(table_path, index=False)
latex_path = fig_dir / "agreement_table_humans_and_models.tex"
agreement_table.to_latex(latex_path, index=False, float_format="%.4f")
print("\nAgreement Table (sorted by average_all_emotions):")
print(agreement_table.to_string(index=False))
print(f"Saved agreement table to {table_path} and {latex_path} (mean agreement per annotator and emotion).")
print("Agreement values are means across emotions; see output for details.")

# --- Summary Table: average + std + median + top-1 emotion ---

summary_rows = []
for _, row in agreement_table.iterrows():
    annotator = row['Annotator']
    values = [row[e] for e in emotion_list if pd.notnull(row[e])]
    avg = np.mean(values) if values else np.nan
    std = np.std(values, ddof=1) if len(values) > 1 else np.nan
    median = np.median(values) if values else np.nan
    # Get top-1 emotion by agreement for this annotator
    emotion_scores = {e: row[e] for e in emotion_list}
    sorted_emotions = sorted(emotion_scores.items(), key=lambda x: (-(x[1] if pd.notnull(x[1]) else -999)),)
    tops = []
    for i in range(5):
        if i < len(sorted_emotions) and pd.notnull(sorted_emotions[i][1]):
            tops.append(f"{sorted_emotions[i][0]} ({sorted_emotions[i][1]:.4f})")
        else:
            tops.append("")
    summary_rows.append({
        "Annotator": annotator,
        "average_all_emotions": avg,
        "std_all_emotions": std,
        "median_all_emotions": median,
        "top-1": tops[0],
        # "top-2": tops[1],
        # "top-3": tops[2],
        # "top-4": tops[3],
        # "top-5": tops[4],
    })
summary_table = pd.DataFrame(summary_rows)
summary_table = summary_table.sort_values("average_all_emotions", ascending=False)

# Save summary table as CSV and LaTeX (NeurIPS style)
summary_csv_path = fig_dir / "agreement_table_summary_humans_and_models.csv"
summary_tex_path = fig_dir / "agreement_table_summary_humans_and_models.tex"
summary_table.to_csv(summary_csv_path, index=False)
summary_table.to_latex(
    summary_tex_path,
    index=False,
    float_format="%.4f",
    columns=[
        "Annotator",
        "average_all_emotions",
        "std_all_emotions",
        "median_all_emotions",
        "top-1", 
        # "top-2", "top-3", "top-4", "top-5"
    ],
    caption="Agreement summary for all annotators: mean, standard deviation, median, and top-5 emotions (with highest agreement) per annotator. For humans, agreement is with all other humans; for models, agreement is with all humans.",
    label="tab:agreement_summary"
)

print("\nSummary Table (NeurIPS style, average/std/median/top-5):")
print(summary_table.to_string(index=False))
print(f"Saved summary table to {summary_csv_path} and {summary_tex_path} (mean, std, median, top-5 emotions per annotator).")
print("Error bars are standard deviation across emotions for each annotator.")

# --- Boxplot: Agreement by Annotator Group ---

# Define annotator group mapping
group_map = {}
for annot in all_annotators:
    if annot in human_annotators:
        group_map[annot] = "Human Annotators"
    elif "Empathic Insight" in annot:
        group_map[annot] = "Our Models"
    elif annot.endswith("MS"):
        group_map[annot] = "VLMs with Multi-Shot Prompt"
    elif annot.endswith("ZS"):
        group_map[annot] = "VLMs with Zero-Shot Prompt"
    elif "Hume Face*" in annot:
        group_map[annot] = "Proprietary Models"
    elif "Random Baseline" in annot:
        group_map[annot] = "Random Baseline"
    else:
        print(f"Warning: Unknown model type for '{annot}'")
        raise Exception(f"Unknown model type: {annot}")

# Prepare data for boxplot: each row is (group, agreement)
boxplot_data = []
for _, row in agreement_table.iterrows():
    annot = row["Annotator"]
    group = group_map.get(annot, "other")
    for emotion in emotion_list:
        val = row[emotion]
        if pd.notnull(val):
            boxplot_data.append({"group": group, "agreement": val})

df_boxplot = pd.DataFrame(boxplot_data)

# Plot boxplot (swapped axes: group on y, agreement on x)
plt.figure(figsize=(10, 3.5))
sns.boxplot(
    data=df_boxplot, y="group", x="agreement",
    order=[
        "Human Annotators", "Our Models", "Proprietary Models",
        "VLMs with Multi-Shot Prompt", "VLMs with Zero-Shot Prompt", "Random Baseline"
    ]
)
plt.title("Agreement by Annotator Group")
plt.ylabel("Annotator Group")
plt.xlabel("Agreement (Weighted Kappa)")
plt.xlim(-0.1, 0.7)
plt.tight_layout()
boxplot_png = fig_dir / "agreement_by_group_boxplot.png"
boxplot_pdf = fig_dir / "agreement_by_group_boxplot.pdf"
plt.savefig(boxplot_png, dpi=600)
plt.savefig(boxplot_pdf, dpi=600)
plt.close()
print(f"Saved agreement by group boxplot to {boxplot_png} and {boxplot_pdf}")
print("Statistical significance between groups is assessed by Mann-Whitney U test;")
print("error bars are 95% bootstrap confidence intervals for mean differences.")

# --- Save pairwise group mean differences and 95% bootstrap CIs to CSV ---

from scipy.stats import mannwhitneyu

group_order = [
    "Human Annotators",
    "Our Models",
    "Proprietary Models",
    "VLMs with Multi-Shot Prompt",
    "VLMs with Zero-Shot Prompt",
    "Random Baseline"
]
pairs = []
for i in range(len(group_order)):
    for j in range(i + 1, len(group_order)):
        pairs.append((group_order[i], group_order[j]))

pairwise_stats = []
for g1, g2 in pairs:
    data1 = df_boxplot[df_boxplot["group"] == g1]["agreement"].dropna()
    data2 = df_boxplot[df_boxplot["group"] == g2]["agreement"].dropna()
    if len(data1) > 1 and len(data2) > 1:
        stat, pval = mannwhitneyu(data1, data2, alternative='two-sided')
        diff = data1.mean() - data2.mean()
        # Bootstrap CI for mean difference
        boot_diffs = []
        n_boot = 1000
        rng = np.random.default_rng(42)
        for _ in range(n_boot):
            boot1 = rng.choice(data1, size=len(data1), replace=True)
            boot2 = rng.choice(data2, size=len(data2), replace=True)
            boot_diffs.append(np.mean(boot1) - np.mean(boot2))
        ci_low, ci_high = np.percentile(boot_diffs, [2.5, 97.5])
        pairwise_stats.append({
            "group1": g1,
            "group2": g2,
            "mean_difference": diff,
            "ci_low": ci_low,
            "ci_high": ci_high,
            "p_value": pval
        })
pairwise_stats_df = pd.DataFrame(pairwise_stats)
pairwise_stats_csv = fig_dir / "agreement_by_group_pairwise_stats.csv"
pairwise_stats_df.to_csv(pairwise_stats_csv, index=False)
print(f"Saved agreement by group pairwise stats to {pairwise_stats_csv}")

# --- Statistical comparison: Are models and humans on par with human annotators? ---

model_human_stats = []

def get_hh_kappas(exclude_human=None):
    """
    Helper: get human-human kappas, excluding a specific human if provided.
    """
    hh_kappas = []
    for h1 in human_annotators:
        if exclude_human is not None and h1 == exclude_human:
            continue
        for h2 in human_annotators:
            if h1 >= h2 or (exclude_human is not None and h2 == exclude_human):
                continue
            for emotion in emotion_list:
                emotion_ratings = df_pivot.loc[emotion]
                if h1 not in emotion_ratings.columns or h2 not in emotion_ratings.columns:
                    continue
                ratings_pair = emotion_ratings[[h1, h2]].dropna()
                if len(ratings_pair) < 2:
                    continue
                ratings1 = ratings_pair[h1].astype(int)
                ratings2 = ratings_pair[h2].astype(int)
                try:
                    kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
                except Exception:
                    kappa = np.nan
                if pd.notnull(kappa):
                    hh_kappas.append(kappa)
    return np.array(hh_kappas)[~np.isnan(hh_kappas)]

# For each model, compare model-human distribution to human-human (all humans)
hh_kappas_all = get_hh_kappas()
for model in model_annotators:
    mh_kappas = []
    for human in human_annotators:
        for emotion in emotion_list:
            emotion_ratings = df_pivot.loc[emotion]
            if model not in emotion_ratings.columns or human not in emotion_ratings.columns:
                continue
            ratings_pair = emotion_ratings[[model, human]].dropna()
            if len(ratings_pair) < 2:
                continue
            ratings1 = ratings_pair[model].astype(int)
            ratings2 = ratings_pair[human].astype(int)
            try:
                kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
            except Exception:
                kappa = np.nan
            if pd.notnull(kappa):
                mh_kappas.append(kappa)
    mh_kappas = np.array(mh_kappas)
    mh_kappas = mh_kappas[~np.isnan(mh_kappas)]

    mean_diff = np.mean(mh_kappas) - np.mean(hh_kappas_all) if len(mh_kappas) > 0 and len(hh_kappas_all) > 0 else np.nan

    n_boot = 1000
    rng = np.random.default_rng(42)
    boot_diffs = []
    for _ in range(n_boot):
        boot_mh = rng.choice(mh_kappas, size=len(mh_kappas), replace=True) if len(mh_kappas) > 0 else np.array([np.nan])
        boot_hh = rng.choice(hh_kappas_all, size=len(hh_kappas_all), replace=True) if len(hh_kappas_all) > 0 else np.array([np.nan])
        boot_diffs.append(np.nanmean(boot_mh) - np.nanmean(boot_hh))
    ci_low, ci_high = np.percentile(boot_diffs, [2.5, 97.5])

    if len(mh_kappas) > 1 and len(hh_kappas_all) > 1:
        stat, pval = mannwhitneyu(mh_kappas, hh_kappas_all, alternative='two-sided')
    else:
        pval = np.nan

    could_be_annotator = "yes" if (pval > 0.05 if not pd.isnull(pval) else False) else "no"

    model_human_stats.append({
        "annotator": model,
        "mean_difference": mean_diff,
        "ci_of_difference": f"[{ci_low:.4f}, {ci_high:.4f}]",
        "p_value": pval,
        "could_be_annotator": could_be_annotator
    })

# For each human, compare their human-vs-human distribution (excluding self) to the rest of humans
for human in human_annotators:
    hvh_kappas = []
    for other in human_annotators:
        if other == human:
            continue
        for emotion in emotion_list:
            emotion_ratings = df_pivot.loc[emotion]
            if human not in emotion_ratings.columns or other not in emotion_ratings.columns:
                continue
            ratings_pair = emotion_ratings[[human, other]].dropna()
            if len(ratings_pair) < 2:
                continue
            ratings1 = ratings_pair[human].astype(int)
            ratings2 = ratings_pair[other].astype(int)
            try:
                kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic', labels=list(range(8)))
            except Exception:
                kappa = np.nan
            if pd.notnull(kappa):
                hvh_kappas.append(kappa)
    hvh_kappas = np.array(hvh_kappas)
    hvh_kappas = hvh_kappas[~np.isnan(hvh_kappas)]

    # human-human kappas for the rest (excluding this human)
    hh_kappas_excl = get_hh_kappas(exclude_human=human)

    mean_diff = np.mean(hvh_kappas) - np.mean(hh_kappas_excl) if len(hvh_kappas) > 0 and len(hh_kappas_excl) > 0 else np.nan

    n_boot = 1000
    rng = np.random.default_rng(42)
    boot_diffs = []
    for _ in range(n_boot):
        boot_hvh = rng.choice(hvh_kappas, size=len(hvh_kappas), replace=True) if len(hvh_kappas) > 0 else np.array([np.nan])
        boot_hh = rng.choice(hh_kappas_excl, size=len(hh_kappas_excl), replace=True) if len(hh_kappas_excl) > 0 else np.array([np.nan])
        boot_diffs.append(np.nanmean(boot_hvh) - np.nanmean(boot_hh))
    ci_low, ci_high = np.percentile(boot_diffs, [2.5, 97.5])

    if len(hvh_kappas) > 1 and len(hh_kappas_excl) > 1:
        stat, pval = mannwhitneyu(hvh_kappas, hh_kappas_excl, alternative='two-sided')
    else:
        pval = np.nan

    could_be_annotator = "yes" if (pval > 0.05 if not pd.isnull(pval) else False) else "no"

    model_human_stats.append({
        "annotator": human,
        "mean_difference": mean_diff,
        "ci_of_difference": f"[{ci_low:.4f}, {ci_high:.4f}]",
        "p_value": pval,
        "could_be_annotator": could_be_annotator
    })

df_model_human_stats = pd.DataFrame(model_human_stats)

# Sort by mean_difference descending
df_model_human_stats = df_model_human_stats.sort_values("mean_difference", ascending=False)
stats_csv_path = fig_dir / "model_vs_human_stats.csv"
stats_tex_path = fig_dir / "model_vs_human_stats.tex"
df_model_human_stats.to_csv(stats_csv_path, index=False)
df_model_human_stats.to_latex(
    stats_tex_path,
    index=False,
    float_format="%.4f",
    columns=["annotator", "mean_difference", "ci_of_difference", "p_value", "could_be_annotator"],
    caption="Statistical comparison of each model and human's agreement with humans vs. the rest of human-human agreement. 'Could be annotator' is 'yes' if p > 0.05.",
    label="tab:model_vs_human_stats"
)
print("\nModel/Human vs Human Annotator Statistical Comparison Table:")
print(df_model_human_stats.to_string(index=False))
print(f"Saved model/human vs human statistical comparison to {stats_csv_path} and {stats_tex_path}.")
print("Mean differences and 95% bootstrap confidence intervals are reported;")
print("p-values from Mann-Whitney U test are included.")

# --- Krippendorff's Alpha: Human Inter-Annotator Reliability ---

try:
    import krippendorff
except ImportError:
    krippendorff = None
    print("krippendorff package not installed. Skipping Krippendorff's alpha plot.")

if krippendorff is not None:
    # Only use human annotators
    df_humans = df_long[df_long['human'] == True]
    alpha_per_emotion = []
    alpha_ci_lows = []
    alpha_ci_highs = []
    alpha_boots_per_emotion = []
    n_boot = 1000
    rng = np.random.default_rng(42)
    for emotion in emotion_list:
        # Build matrix: rows=images, columns=annotators, values=rating
        pivot = df_humans[df_humans['emotion'] == emotion].pivot_table(
            index="image", columns="annotator", values="rating"
        )
        # Build 2D numpy array (annotators x items), np.nan for missing
        data = pivot.T.values
        # Only keep annotators with at least 2 ratings
        data = [row for row in data if np.count_nonzero(~np.isnan(row)) > 1]
        if len(data) < 2:
            alpha = None
            ci_low = None
            ci_high = None
            boot_alphas = []
        else:
            data_arr = np.array(data)
            alpha = krippendorff.alpha(reliability_data=data_arr, level_of_measurement='interval')
            # Bootstrap CI over images (columns)
            boot_alphas = []
            n_items = data_arr.shape[1]
            for _ in range(n_boot):
                # Sample images (columns) with replacement
                idxs = rng.choice(n_items, size=n_items, replace=True)
                boot_data = data_arr[:, idxs]
                try:
                    boot_alpha = krippendorff.alpha(reliability_data=boot_data, level_of_measurement='interval')
                except Exception:
                    boot_alpha = np.nan
                boot_alphas.append(boot_alpha)
            boot_alphas = np.array([a for a in boot_alphas if not pd.isnull(a)])
            if len(boot_alphas) > 0:
                ci_low, ci_high = np.percentile(boot_alphas, [2.5, 97.5])
            else:
                ci_low, ci_high = np.nan, np.nan
        alpha_per_emotion.append(alpha)
        alpha_ci_lows.append(ci_low)
        alpha_ci_highs.append(ci_high)
        alpha_boots_per_emotion.append(boot_alphas)
    # Shorten emotion labels: keep only part after "|"
    def short_label(e):
        return e.split("|")[-1].strip()
    short_emotion_labels = [short_label(emotion_list[i]) for i in range(len(emotion_list))]
    # Sort by alpha descending
    sorted_indices = np.argsort(alpha_per_emotion)[::-1]
    sorted_emotions = [short_emotion_labels[i] for i in sorted_indices]
    sorted_boots = [alpha_boots_per_emotion[i] for i in sorted_indices]
    sorted_alphas = [alpha_per_emotion[i] for i in sorted_indices]
    sorted_ci_lows = [alpha_ci_lows[i] for i in sorted_indices]
    sorted_ci_highs = [alpha_ci_highs[i] for i in sorted_indices]

    # Boxplot for Krippendorff's alpha bootstrap distribution per emotion
    import matplotlib.pyplot as plt
    import numpy as np
    plt.figure(figsize=(10, max(8, len(emotion_list)*0.35)))
    box_data = sorted_boots
    plt.boxplot(
        box_data,
        vert=False,
        patch_artist=True,
        labels=sorted_emotions,
        showmeans=False,
        boxprops=dict(facecolor='lightblue', color='C0'),
        medianprops=dict(color='C0', linewidth=2),
        whiskerprops=dict(color='C0'),
        capprops=dict(color='C0'),
        flierprops=dict(markerfacecolor='gray', marker='o', markersize=3, alpha=0.3)
    )
    # Save alpha summary to CSV
    alpha_box_csv = fig_dir / "krippendorff_alpha_per_emotion_boxplot.csv"
    alpha_box_df = pd.DataFrame({
        "emotion": sorted_emotions,
        "alpha": sorted_alphas,
        "ci_low": sorted_ci_lows,
        "ci_high": sorted_ci_highs
    })
    # Add mean and median rows
    mean_row = pd.DataFrame([{
        "emotion": "mean",
        "alpha": np.nanmean(sorted_alphas),
        "ci_low": np.nanmean(sorted_ci_lows),
        "ci_high": np.nanmean(sorted_ci_highs)
    }])
    median_row = pd.DataFrame([{
        "emotion": "median",
        "alpha": np.nanmedian(sorted_alphas),
        "ci_low": np.nanmedian(sorted_ci_lows),
        "ci_high": np.nanmedian(sorted_ci_highs)
    }])
    alpha_box_df = pd.concat([alpha_box_df, mean_row, median_row], ignore_index=True)
    alpha_box_df = alpha_box_df.sort_values("alpha", ascending=False)
    alpha_box_df.to_csv(alpha_box_csv, index=False)
    print(f"Saved Krippendorff's alpha boxplot data to {alpha_box_csv}")

    plt.xlabel("Krippendorff's α (interval)")
    plt.ylabel("Emotion")
    plt.title("Human Inter-Annotator Reliability (Krippendorff's α)")
    plt.tight_layout()
    alpha_box_png = fig_dir / "_krippendorff_alpha_per_emotion_boxplot.png"
    alpha_box_pdf = fig_dir / "_krippendorff_alpha_per_emotion_boxplot.pdf"
    plt.savefig(alpha_box_png, dpi=1200)
    plt.savefig(alpha_box_pdf, dpi=1200)
    plt.close()
    print(f"Saved Krippendorff's alpha boxplot to {alpha_box_png} and {alpha_box_pdf}")

# --- Model vs Human Consensus: Spearman Only, Sorted Barplots with Error Bars ---

from scipy.stats import spearmanr

metrics = ["spearman_rho"]
agg_types = ["median"]
model_human_metrics = {agg: [] for agg in agg_types}

# Define annotator group mapping for bar textures
group_map = {}
for annot in all_annotators:
    if annot in human_annotators:
        group_map[annot] = "Human Annotators"
    elif "Empathic Insight" in annot:
        group_map[annot] = "Our Models"
    elif annot.endswith("MS"):
        group_map[annot] = "VLMs with Multi-Shot Prompt"
    elif annot.endswith("ZS"):
        group_map[annot] = "VLMs with Zero-Shot Prompt"
    elif "Hume Face*" in annot:
        group_map[annot] = "Proprietary Models"
    elif "Random Baseline" in annot:
        group_map[annot] = "Random Baseline"
    else:
        print(f"Warning: Unknown model type for '{annot}'")
        raise Exception(f"Unknown model type: {annot}")

# Define hatches for each group
hatch_map = {
    "Human Annotators": "",
    "Our Models": "**",
    "VLMs with Multi-Shot Prompt": "x",
    "VLMs with Zero-Shot Prompt": "..",
    "Random Baseline": "++",
    "Proprietary Models": "//"
}

for agg in agg_types:
    for model in model_annotators:
        for emotion in emotion_list:
            emotion_ratings = df_pivot.loc[emotion]
            if model not in emotion_ratings.columns:
                row = {"model": model, "emotion": emotion}
                for m in metrics:
                    row[m] = None
                model_human_metrics[agg].append(row)
                continue
            model_ratings = emotion_ratings[model]
            human_cols = [h for h in human_annotators if h in emotion_ratings.columns]
            if not human_cols:
                row = {"model": model, "emotion": emotion}
                for m in metrics:
                    row[m] = None
                model_human_metrics[agg].append(row)
                continue
            if agg == "mean":
                human_agg = emotion_ratings[human_cols].mean(axis=1)
            else:
                human_agg = emotion_ratings[human_cols].median(axis=1)
            valid_idx = model_ratings.dropna().index.intersection(human_agg.dropna().index)
            if len(valid_idx) < 2:
                row = {"model": model, "emotion": emotion}
                for m in metrics:
                    row[m] = None
                model_human_metrics[agg].append(row)
                continue
            mvals = model_ratings.loc[valid_idx]
            hvals = human_agg.loc[valid_idx]
            # Spearman correlation
            try:
                rho, _ = spearmanr(mvals, hvals)
            except Exception:
                rho = None
            model_human_metrics[agg].append({
                "model": model,
                "emotion": emotion,
                "spearman_rho": rho
            })

# Aggregate across all emotions for each model and plot sorted barplots with error bars and textures
n_boot = 1000
rng = np.random.default_rng(42)
for agg in agg_types:
    df_metrics = pd.DataFrame(model_human_metrics[agg])
    for metric in metrics:
        # Aggregate: mean and bootstrap CI across all emotions for each model
        agg_df = df_metrics.groupby("model")[metric].agg(['mean']).reset_index()
        # Bootstrap 95% CI for mean
        ci_lows = []
        ci_highs = []
        for model in agg_df["model"]:
            vals = df_metrics[df_metrics["model"] == model][metric].dropna().values
            if len(vals) < 2:
                ci_lows.append(np.nan)
                ci_highs.append(np.nan)
                continue
            boots = [rng.choice(vals, size=len(vals), replace=True).mean() for _ in range(n_boot)]
            ci_low, ci_high = np.percentile(boots, [2.5, 97.5])
            ci_lows.append(ci_low)
            ci_highs.append(ci_high)
        agg_df["ci_low"] = ci_lows
        agg_df["ci_high"] = ci_highs
        # Sort by mean descending
        agg_df = agg_df.sort_values("mean", ascending=False)
        # Get group and hatch for each model
        agg_df["group"] = agg_df["model"].map(group_map).fillna("other")
        agg_df["hatch"] = agg_df["group"].map(hatch_map).fillna("")
        # Plot with error bars and textures
        plt.figure(figsize=(max(8, len(model_annotators)*0.5), 5))

        bars = plt.bar(
            agg_df["model"], agg_df["mean"], color='#7ec8e3',
            yerr=[agg_df["mean"]-agg_df["ci_low"], agg_df["ci_high"]-agg_df["mean"]],
            capsize=6
        )
        # Apply hatches
        for bar, hatch in zip(bars, agg_df["hatch"]):
            bar.set_hatch(hatch)
        # Legend for textures
        legend_handles = []
        seen = set()
        import matplotlib.patches as mpatches
        for group, hatch in hatch_map.items():
            if group in agg_df["group"].values and group not in seen:
                patch = mpatches.Patch(facecolor='#7ec8e3', hatch=hatch, label=group)
                legend_handles.append(patch)
                seen.add(group)
        plt.legend(handles=legend_handles, title="Annotator Group", loc="upper right")
        plt.ylabel(metric.replace("_", " ").title())
        plt.xlabel("Model Annotator")
        plt.title(f"Model vs Human ({agg.title()} Aggregation): Mean {metric.replace('_', ' ').title()} Across All Emotions")
        plt.ylim(-0.1, 0.75)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        fname = f"_model_vs_human_{agg}_{metric}_aggregate_barplot"
        plt.savefig(fig_dir / f"{fname}.png", dpi=300)
        plt.savefig(fig_dir / f"{fname}.pdf", dpi=300)
        plt.close()
        print(f"Saved aggregate {metric} barplot ({agg}) to {fig_dir / (fname + '.png')} and .pdf")